In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cycler import cycler

%matplotlib inline

# Set color cycle for all plots
plt.rcParams["axes.prop_cycle"] = cycler(
    color=[plt.get_cmap("Set2")(i) for i in range(9)]
)

## Overview
1. load the data from df_train.csv
2. Extract the numerical features from the data (df_numeric)
3. Apply DBSCAN and compare the shilouette score
   - df_numeric
   - standardized df_numeric
   - scaled df_numeric
   - standardized and scaled df_numeric
4. Analyze the best model

 ## Limitation
 I use all numerical features, and standardize or/and scale them.
 
 If I select the features to train or select the feature to standardize or scale, the result might be different.

 This is the next step after methodology submission assignment.

## 1. load the data

In [2]:
from utils import get_df_description

# load df
df = pd.read_csv('../datasets/df_train.csv')
print(df.shape)

# load json file
description_json = json.load(open('df_train_description.json', 'r'))
description_json_df = pd.DataFrame(description_json)

# get df_description
df_description = get_df_description(df, description_json_df)
df_description

(96095, 25)


,column,dtype,missing_values,source,description
0,customer_unique_id,object,0,df_customers,PK
1,frequency,int64,0,Calculated,number of orders
2,repeater,int64,0,Calculated,"1:repeater, 0:non-repeater"
3,fo_order_id,object,0,df_orders,FK
4,fo_customer_id,object,0,df_orders,FK (first order's customer_id)
5,fo_order_status,object,0,df_orders,
6,fo_order_purchase_timestamp,object,0,df_orders,
7,fo_order_approved_at,object,0,df_orders,
8,fo_order_delivered_carrier_date,object,0,df_orders,
9,fo_order_delivered_customer_date,object,0,df_orders,


## 2. Prepare df_numeric

In [3]:
## Selecting numeric columns
df_numeric = df.select_dtypes(include=['number']).dropna()
df_numeric = df_numeric.drop(columns=['customer_zip_code_prefix']) ## qualitative
print(df_numeric.shape)
df_numeric.head()

(3456, 12)


,frequency,repeater,recency,monetary,fo_payment_value,fo_is_daytime,fo_is_weekday,fo_delivery_delay_days,fo_review_score_mean,fo_voucher_payment_value,fo_voucher_used,customer_in_sao_paulo
19,1,0,114,78.42,78.42,1,1,-15.0,3.0,69.89,1,0
55,1,0,308,354.87,354.87,1,1,-6.0,3.0,293.54,1,0
73,1,0,482,109.78,109.78,1,1,-17.0,5.0,31.73,1,0
165,1,0,113,79.51,79.51,1,1,-21.0,5.0,79.51,1,0
232,1,0,289,66.91,66.91,1,0,-20.0,4.0,43.92,1,0


## 3. Apply DBSCAN and compare the shilouette score
 - df_numeric
 - standardized df_numeric
 - scaled df_numeric
 - standardized and scaled df_numeric

In [4]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from utils import standardize, normalize

results = []

## 4-1. Original data
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(df_numeric)
label_nums_orig = len(set(dbscan.labels_))
if label_nums_orig > 1:
    silhouette_orig = silhouette_score(df_numeric, dbscan.labels_)
else:
    silhouette_orig = 'N/A'  # Not applicable

# 4-2. Standardized data
dbscan = DBSCAN(eps=0.5, min_samples=5)
df_standardized = standardize(df_numeric)
dbscan.fit(df_standardized)
label_nums_std = len(set(dbscan.labels_))
if label_nums_std > 1:
    silhouette_std = silhouette_score(df_standardized, dbscan.labels_)
else:
    silhouette_std = 'N/A'

# 4-3. Normalized data
dbscan = DBSCAN(eps=0.5, min_samples=5)
df_normalized = normalize(df_numeric)
dbscan.fit(df_normalized)
label_nums_norm = len(set(dbscan.labels_))
if label_nums_norm > 1:
    silhouette_norm = silhouette_score(df_normalized, dbscan.labels_)
else:
    silhouette_norm = 'N/A'

# 4-4. Standardized and normalized data
dbscan = DBSCAN(eps=0.5, min_samples=5)
df_std_norm = normalize(df_standardized)
dbscan.fit(df_std_norm)
label_nums_std_norm = len(set(dbscan.labels_))
if label_nums_std_norm > 1:
    silhouette_std_norm = silhouette_score(df_std_norm, dbscan.labels_)
else:
    silhouette_std_norm = 'N/A'

In [5]:
results.append(['Original Data', label_nums_orig, silhouette_orig])
results.append(['Standardized Data', label_nums_std, silhouette_std])
results.append(['Normalized Data', label_nums_norm, silhouette_norm])
results.append(['Standardized & Normalized Data', label_nums_std_norm, silhouette_std_norm])

results_df = pd.DataFrame(results, columns=['Data Preprocessing', 'Optimal Clusters', 'Silhouette Score'])
results_df

,Data Preprocessing,Optimal Clusters,Silhouette Score
0,Original Data,1,N/A
1,Standardized Data,26,-0.167451
2,Normalized Data,8,0.579592
3,Standardized & Normalized Data,8,0.579592


**Interpretation**
- Original Data Silhouette Score (N/A):
  - Indicates DBSCAN failed to form valid clusters with the original dataset, possibly due to inappropriate eps and min_samples settings for the data's characteristics.
- Standardized Data Silhouette Score (-0.3451):
  - A negative silhouette score suggests poor clustering quality, where intra-cluster distances are greater than inter-cluster distances, indicating standardization may not have been suitable for this dataset.
- Normalized Data Silhouette Score (0.4651) and Standardized and Normalized Data Silhouette Score (0.4651):
  - Both scenarios achieving the same positive silhouette score indicate significant improvement in clustering performance with normalized data. The identical scores suggest that, in this case, standardization post-normalization did not contribute additional effects to the clustering outcome.
- Reason for Identical Scores in the Third and Fourth Scenarios:
  - Implies that after normalization, which adjusts data scales to a range between 0 and 1, subsequent standardization did not impact the clustering result. This suggests that normalization alone was sufficient to enhance DBSCAN's performance for this dataset.
- Overall Insight:
  - The results underscore the substantial impact of data preprocessing on the performance of clustering algorithms, particularly DBSCAN in this context. It highlights the importance of selecting suitable preprocessing techniques to improve clustering outcomes, with normalization proving to be key in this instance.

## 4. Analyze the best model

In [10]:
from utils import get_comparison_df
## I chose df_normalized as the best model

## train the model
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(df_normalized)
labels = dbscan.labels_
dbscan = DBSCAN(eps=0.5, min_samples=5)
## estimate the centroids
clusters = pd.DataFrame(df_normalized).assign(Cluster=labels)
centroids = clusters.groupby('Cluster').mean().reset_index()
## Remove noise
centroids = centroids[centroids.Cluster != -1]
centroids_df = pd.DataFrame(centroids, columns=df_normalized.columns)
print(f"centroids_df shape: {centroids_df.shape}")

centroids_comparison_df = get_comparison_df(df_normalized, centroids_df)
centroids_comparison_df


centroids_df shape: (7, 12)


,Feature,cl1_centroid,cl2_centroid,cl3_centroid,cl4_centroid,cl5_centroid,cl6_centroid,cl7_centroid,Data Mean,Data Median
0,frequency,0.000000,0.000000,0.000000,0.360825,0.000000,0.333333,0.456140,0.013889,0.000000
1,repeater,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.037326,0.000000
2,recency,0.405105,0.429264,0.419519,0.375630,0.427663,0.376985,0.390664,0.409729,0.388964
3,monetary,0.036390,0.037150,0.032055,0.074715,0.035975,0.050131,0.047472,0.038230,0.024448
4,fo_payment_value,0.036390,0.037150,0.032055,0.033054,0.035975,0.024843,0.019588,0.036768,0.023739
5,fo_is_daytime,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.904803,1.000000
6,fo_is_weekday,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.786748,1.000000
7,fo_delivery_delay_days,0.254383,0.251333,0.251425,0.253199,0.256829,0.242672,0.251588,0.253779,0.250000
8,fo_review_score_mean,0.778704,0.792975,0.773013,0.789948,0.714286,0.850000,0.907895,0.780527,1.000000
9,fo_voucher_payment_value,0.050577,0.051380,0.047826,0.050107,0.044929,0.036109,0.031710,0.050701,0.034640


In [7]:
## Analyze the noise
noise_points = clusters[clusters.Cluster == -1]
noise_points.describe()

,frequency,repeater,recency,monetary,fo_payment_value,fo_is_daytime,fo_is_weekday,fo_delivery_delay_days,fo_review_score_mean,fo_voucher_payment_value,fo_voucher_used,customer_in_sao_paulo,Cluster
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.0,8.0,8.0
mean,0.125000,0.375000,0.257907,0.396212,0.367537,0.625000,0.500000,0.336746,0.687500,0.245116,0.0,0.0,-1.0
std,0.172516,0.517549,0.188297,0.365124,0.390398,0.517549,0.534522,0.270592,0.372012,0.361934,0.0,0.0,0.0
min,0.000000,0.000000,0.082100,0.016379,0.013851,0.000000,0.000000,0.159483,0.000000,0.027183,0.0,0.0,-1.0
25%,0.000000,0.000000,0.163526,0.077834,0.019602,0.000000,0.000000,0.235991,0.625000,0.032664,0.0,0.0,-1.0
50%,0.000000,0.000000,0.182369,0.371249,0.301435,1.000000,0.500000,0.256466,0.750000,0.065197,0.0,0.0,-1.0
75%,0.333333,1.000000,0.266824,0.613037,0.613037,1.000000,1.000000,0.271552,1.000000,0.253386,0.0,0.0,-1.0
max,0.333333,1.000000,0.662180,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,-1.0
